In [2]:
import os
import re

from datasets import load_dataset
import pandas as pd

import requests
import json

from statistics import mean

import random
import csv
import nltk
from nltk.corpus import gutenberg
import numpy as np

import random

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Presidential Speeches - Data

In [37]:
url = "https://millercenter.org/sites/default/files/corpus/presidential-speeches.json"
filename = "./data/Speeches/presidential-speeches.json"

# Get request to the url
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Get the JSON data from the response
    json_data = response.json()

    # Save the JSON data to a file
    with open(filename, "w") as file:
        json.dump(json_data, file)

    print(f"JSON data saved to {filename} successfully.")
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")


JSON data saved to ./data/Speeches/presidential-speeches.json successfully.


In [38]:
with open('./data/Speeches/presidential-speeches.json', 'r') as f:
  data = json.load(f)

speech_texts = [element["transcript"] for element in data if len(element["transcript"].split())>=1100]
print(len(speech_texts))
for i in range(len(speech_texts)):
    speech_texts[i] = speech_texts[i].replace("\r\n\r\n", " ")
    speech_texts[i] = speech_texts[i].replace("\r\n", "")
    if (len(speech_texts[i]))<=1400:
       print(len(speech_texts[i]))

801


In [39]:
speech_texts_short = []

for text in speech_texts:
    target_word_count = random.randint(500, 800)
    total_word_count = 0
    selected_sentences = []
    sentences = nltk.sent_tokenize(text)

    sentence_length = []
    for sentence in sentences:
        sentence_length.append(len(sentence.split()))

    max_skip = (len(sentences)*int(mean(sentence_length)))-target_word_count
    words_to_skip = random.randint(0, max_skip)
    for sentence in sentences:
        words = sentence.split()

        if total_word_count + len(words) <= words_to_skip:
            total_word_count += len(words)
            continue
        if total_word_count + len(words) <= target_word_count + words_to_skip:
            selected_sentences.append(sentence)
            total_word_count += len(words)
        else:
            break
    speech_texts_short.append(' '.join(selected_sentences))

In [40]:
speech_class = ['Political speech'] * len(speech_texts_short)

df_speeches = pd.DataFrame({'Text': speech_texts_short, 'Class': speech_class})

In [41]:
print(speech_texts_short[0])

These reductions come as a result of the cost-cutting programs which I asked each agency head to put into effect last November and December. The results represent some progress in our drive to raise the efficiency of the Federal Government and to cut the cost. Details will be available from Mr. Salinger. Today I have a report on the first results of our efforts to reduce the cost of Government publications. With only a few agencies reporting, with the bulk of the work yet to be done over the next several months, it is gratifying to note that already we have eliminated 158 existing or proposed publications for savings of more than $1 million to the taxpayers. I will be glad to take any questions. Q. Mr President, Soviet officials have told an American delegation that they would like to sign a long-term trade agreement with the United States. Do you favor more wheat sales to the Soviet Union, and do you favor a long-term trade agreement with the U.S.S.R.? THE PRESIDENT. We would be very 

In [42]:
# Print the average number of words in each text
array = []
for transcript in speech_texts_short:
    words = transcript.split()
    num_words = len(words)
    array.append(num_words)
print(mean(array))

644.7315855181024


# CNN News - Data

In [43]:
# Load the News-Dataset from Huggingface
news_dataset = load_dataset("cnn_dailymail", "3.0.0")

Found cached dataset cnn_dailymail (/Users/bnnlukas/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

In [44]:
# Get the first 1000 texts of the dataset
dataset_news = news_dataset['train']
news_texts = dataset_news['article'][:1100]

In [45]:
news_texts_long = []
for text in news_texts:
    if len(text.split()) >= 250:
        news_texts_long.append(text)
print(news_texts_long[5])

BAGHDAD, Iraq (CNN) -- Dressed in a Superman shirt, 5-year-old Youssif held his sister's hand Friday, seemingly unaware that millions of people across the world have been touched by his story. Nearby, his parents talked about the new future and hope they have for their boy -- and the potential for recovery from his severe burns. Youssif holds his sister's hand Friday. He's wearing a facial mask often used to help burn victims. It's the best birthday present the Iraqi family could ever have imagined for their boy: Youssif turns 6 next Friday. "I was so happy I didn't know what to do with myself," his mother, Zainab, told CNN, a broad smile across her face. "I didn't think the reaction would be this big." His father said he was on the roof of his house when CNN called him with the news about the outpouring of support for his son. "We just want to thank everyone who has come forward," he said. "We knew there was kindness out there." Like his wife, he couldn't stop smiling. He talked about

In [65]:
# Cut out the first sequuence ('BAGHDAD, Iraq (CNN) --')
news_texts_modified = []
for text in news_texts_long:
    parts = text.split('--', 1)
    if len(parts) > 1:
        news_texts_modified.append(parts[1])  # Take the second part after the split
    else:
        news_texts_modified.append(text)  # Keep the original string if '--' is not found

In [67]:
print(news_texts_modified[5])

 Dressed in a Superman shirt, 5-year-old Youssif held his sister's hand Friday, seemingly unaware that millions of people across the world have been touched by his story. Nearby, his parents talked about the new future and hope they have for their boy -- and the potential for recovery from his severe burns. Youssif holds his sister's hand Friday. He's wearing a facial mask often used to help burn victims. It's the best birthday present the Iraqi family could ever have imagined for their boy: Youssif turns 6 next Friday. "I was so happy I didn't know what to do with myself," his mother, Zainab, told CNN, a broad smile across her face. "I didn't think the reaction would be this big." His father said he was on the roof of his house when CNN called him with the news about the outpouring of support for his son. "We just want to thank everyone who has come forward," he said. "We knew there was kindness out there." Like his wife, he couldn't stop smiling. He talked about how he tried in vain 

In [68]:
news_class = ['News'] * len(news_texts_modified)

df_news = pd.DataFrame({'Text': news_texts_modified, 'Class': news_class})

In [69]:
# Print the average number of words in each text
array = []
for text in news_texts_modified:
    words = text.split()
    num_words = len(words)
    array.append(num_words)
print(mean(array))

628.3033932135728


# Jurisdictions - Data

In [48]:
data_path_jurisdictions = './data/Juristictions/'

jurisdictions_texts = []
for file_name in os.listdir(data_path_jurisdictions):
    file_path = os.path.join(data_path_jurisdictions, file_name)
    with open(file_path, 'r', encoding='latin-1') as file:
        text = file.read()
        if len(text.split()) <=3000:
            continue
        else:
            jurisdictions_texts.append(text)

In [49]:
jurisdictions_texts_short = []

for text in jurisdictions_texts:
    target_word_count = random.randint(500, 800)
    total_word_count = 0
    selected_sentences = []
    sentences = nltk.sent_tokenize(text)
    sentence_length = []
    for sentence in sentences:
        sentence_length.append(len(sentence.split()))

    max_skip = (len(sentences)*int(mean(sentence_length)))-target_word_count
    words_to_skip = random.randint(0, max_skip)
    for sentence in sentences:
        words = sentence.split()

        if total_word_count + len(words) <= words_to_skip:
            total_word_count += len(words)
            continue
        if total_word_count + len(words) <= target_word_count + words_to_skip:
            selected_sentences.append(sentence)
            total_word_count += len(words)
        else:
            break
    jurisdictions_texts_short.append(' '.join(selected_sentences))

In [50]:
jurisdictions_class = ['Jurisdiction'] * len(jurisdictions_texts_short)

# Create the Dataframe for the jurisdictions
df_jurisdictions = pd.DataFrame({'Text': jurisdictions_texts_short, 'Class': jurisdictions_class})

array_jurisdictions = []
for text in jurisdictions_texts_short:
    words = text.split()
    num_words = len(words)
    array_jurisdictions.append(num_words)

# Print the average number of words in each text
print(mean(array_jurisdictions))

# Print the first rows of the dataframe
df_jurisdictions.head()

651.5628205128205


,Text,Class
0,"As to the alleged conspiracy, the essence of t...",Jurisdiction
1,One can have reservations about a policy frame...,Jurisdiction
2,Under rule 54(2) the employment tribunal or ju...,Jurisdiction
3,The practical importance of that principle was...,Jurisdiction
4,Relying on that recommendation and the CJEUs r...,Jurisdiction


# Literature - Data

In [3]:
anzahl_texte = 1000
buch_ids = []

for i in range(anzahl_texte):
        buch = random.choice(gutenberg.fileids())
        buch_ids.append(buch)

file_path = "./data/Literature/buch_ids.txt"

with open(file_path, "w") as file:
    for item in buch_ids:
        file.write(str(item) + "\n")

In [ ]:
file_path = "./data/Literature/buch_ids.txt"
ausgabe_datei = './data/Literature/literarische_texte.csv'

with open(file_path, "r") as file:
    buch_ids = file.read().splitlines()

header = ['Titel', 'Text']

with open(ausgabe_datei, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(header)

    for buch in buch_ids:
        titel = buch.replace('.txt', '')
        text = gutenberg.raw(buch)
        writer.writerow([titel, text])

In [20]:
df_lit = pd.read_csv('./data/Literature/literarische_texte.csv')

In [21]:
literature_texts = []
for text in df_lit['Text']:
    cleaned_text = re.sub(r'^\[[^\]]+\]\s*', '', text)
    cleaned_text = cleaned_text.replace('\n', ' ')
    # Replace all multispaces with singlespaces to get texts that are more clean
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    literature_texts.append(cleaned_text)

print(len(literature_texts))

18


In [54]:
literature_texts_short = []

for text in literature_texts:
    target_word_count = random.randint(500, 800)
    total_word_count = 0
    selected_sentences = []
    sentences = nltk.sent_tokenize(text)       
    sentence_length = []
    for sentence in sentences:
        sentence_length.append(len(sentence.split()))

    max_skip = (len(sentences)*int(mean(sentence_length)))-target_word_count
    words_to_skip = random.randint(0, max_skip)
    for sentence in sentences:
        words = sentence.split()

        if total_word_count + len(words) <= words_to_skip:
            total_word_count += len(words)
            continue
        if total_word_count + len(words) <= target_word_count + words_to_skip:
            selected_sentences.append(sentence)
            total_word_count += len(words)
        else:
            break
    literature_texts_short.append(' '.join(selected_sentences))

In [55]:
literature_class = ['Literature'] * len(literature_texts_short)

# Create the Dataframe for the literature texts
df_literature = pd.DataFrame({'Text': literature_texts_short, 'Class': literature_class})

# Print the first rows of the dataframe
df_literature.head()

,Text,Class
0,"It contained these words: ""DEAR MR. OAKLY,--I ...",Literature
1,"Mr. and Mrs. Elton, indeed, shewed no unwillin...",Literature
2,"Often, when forced from his hammock by exhaust...",Literature
3,The Queene (my Lord) is dead Macb. She should ...,Literature
4,I see poor crazed Lucia's eyes' unnatural glea...,Literature


In [56]:
# Print the average number of words in each text
array = []
for text in literature_texts_short:
    words = text.split()
    num_words = len(words)
    array.append(num_words)
print(mean(array))

650.693


# Blogs - Data

In [57]:
# Data-Source: https://www.kaggle.com/datasets/rtatman/blog-authorship-corpus
df_blo = pd.read_csv('./data/Blogs/blogtext.csv')

In [58]:
n=0
# Create array with blogs over 1000 words to cut too short blogs
large_blog_texts = []
array = []
for text in df_blo['text']:
    words = text.split()
    num_words = len(words)
    if 1000 <= num_words <= 1500 and n < 1000:
        n+=1
        large_blog_texts.append(text)
        array.append(num_words)
# Print the average number of words in each text
print(n)
print(mean(array))

1000
1184.917


In [59]:
target_word_count = 650

# Skip the first 200 words to just get clear text and not the title, the authors, the copyright, etc.
words_to_skip = 200
blog_texts = []

for text in large_blog_texts:
    total_word_count = 0
    selected_sentences = []
    sentences = nltk.sent_tokenize(text)
    for sentence in sentences:
        words = sentence.split()

        if total_word_count + len(words) <= words_to_skip:
            total_word_count += len(words)
            continue
        if total_word_count + len(words) <= target_word_count + words_to_skip:
            selected_sentences.append(sentence)
            total_word_count += len(words)
        else:
            break
    if len(' '.join(selected_sentences).split()) >= 250:
        blog_texts.append(' '.join(selected_sentences))

In [60]:
blog_class = ['Blog'] * len(blog_texts)

# Create the dataframe for the blog texts
df_blogs = pd.DataFrame({'Text': blog_texts, 'Class': blog_class})

# Print the first rows of the dataframe
df_blogs.head()

,Text,Class
0,"i saw one (two, actually, but one appeared to ...",Blog
1,I can't help it HIM (4:38:07 PM): guess you do...,Blog
2,Italian-Australian immigrants always harken ba...,Blog
3,"Your whole family is a bit embarrassed, but yo...",Blog
4,"We each buy nothing, coming back to the realit...",Blog


# Result Dataframe

In [78]:
# Create the resulting dataframe for all texts with the related Classes
result_df = pd.concat([df_speeches, df_news, df_jurisdictions, df_literature, df_blogs], ignore_index=True)

# Print the first rows of the resulting dataframe 
result_df.head()

,Text,Class
0,These reductions come as a result of the cost-...,Political speech
1,"Q. Mr. President, there have been some conflic...",Political speech
2,Now there are going to be some very serious pr...,Political speech
3,"And so far as I am concerned, I hope to keep a...",Political speech
4,"Others walk that road today, and our experienc...",Political speech


In [79]:
print(len(result_df))

4581


In [80]:
for index, row in result_df.iterrows():
    text = row['Text']
    words = text.split()
    if len(words) < 230:
        result_df = result_df.drop(index)

In [84]:
from statistics import mean
# Print the average number of words in each text
array = []
for text in result_df['Text']:
    words = text.split()
    num_words = len(words)
    array.append(num_words)
print(mean(array))

645.1966353506664


In [64]:
# Export the dataframe to a csv-File
result_df.to_csv('./data/Result/dataset.csv', index=False)